<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/en-ko-translator-llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_path = 'drive/MyDrive/llm-study/'

In [4]:
# GPU 사용 체크
# TPU를 사용하는 경우 !nvidia-smi 명령어는 작동하지 않습니다.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [5]:
# 고용량 메모리 사용 체크
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


데이터 토큰화

In [12]:
import torch
torch.__version__ # 2.3.0+cu121

'2.3.0+cu121'

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [14]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [10]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [17]:
import multiprocessing

In [18]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

8

In [ ]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)

In [ ]:
# model_ckpt = "KETI-AIR/ke-t5-base"
# device = "cuda"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [22]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import evaluate
evaluate.__version__ # 0.4.2

'0.4.2'

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import sacrebleu
sacrebleu.__version__ # 2.4.2

'2.4.2'

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
np.__version__ # 1.25.2

'1.25.2'

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

In [ ]:
from transformers import EarlyStoppingCallback, Seq2SeqTrainingArguments

In [ ]:
import transformers
transformers.__version__ # 4.41.0

'4.41.0'

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import accelerate
accelerate.__version__

'0.30.1'

In [ ]:
!pip install git+https://github.com/huggingface/accelerate # 이게 해결법이 되엇다!

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-y9njprfc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-y9njprfc
  Resolved https://github.com/huggingface/accelerate to commit 1cb6fdcf7be5f118d27f177bbbf7053923e41c3e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.31.0.dev0-py3-none-any.whl size=307371 sha256=c5b0258b791fc17c39771db2be9e30c2e90192d75c962da93db38736aa3b663c
  Stored in directory: /tmp/pip-ephem-wheel-cache-huolygzu/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [ ]:
# 새로운 TrainingArguments 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=drive_path+"chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=10,  # 에폭 수를 더 크게 설정
    save_steps=500,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

In [ ]:
# !pip install transformers[torch]

In [ ]:
# !pip install --upgrade transformers[torch]
# !pip install --upgrade accelerate

In [23]:
from transformers import AutoModelForSeq2SeqLM

In [24]:
model_dir = drive_path+"chkpt/checkpoint-56250"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu()

T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [54]:
input_text = [
    "I got my peaches out in Georgia (oh, yeah, shit)",
    "I'm hungry. But I won't eat. Because I'm on a diet. I have to exercise, but it's annoying. I just want to sleep.",
    """I've been searching, but I just don't see the signs""",
    """Look up the name and net income for each company.""",
    "give me data about seasonal net profit and sales for each company"
]

In [55]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

In [56]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

In [57]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 조지아에서 복숭아를 빼냈어요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 배고프지만 먹지 않을 거야, 다이어트 중이라 운동해야 하는데 귀찮아서 그냥 자고 싶어.</s>',
 '<pad> 수색하고 있지만, 간판이 보이지 않을 뿐이에요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 각 회사의 이름과 순이익을 찾아보세요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 각 회사의 계절별 순이익과 매출에 대한 자료를 주세요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [ ]:
new_trainer.save_model("./new_results")